# Global

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/MyDrive/Try_Robotics2_project/"
except:
    root_path = "./"

In [3]:
global opt_update, get_params, nn_forward_fn
global best_params, best_loss

# Imports

In [4]:
import argparse
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

from copy import deepcopy as copy
from functools import partial
from jax import jit
from jax import random
from jax.example_libraries import stax
from jax.example_libraries.stax import serial, Dense, Softplus, Tanh, elementwise, Relu
from jax.example_libraries import optimizers
from jax.experimental.ode import odeint
from jax.tree_util import tree_flatten
from tqdm import tqdm

# Utils

In [5]:
def wrap_coords(state):
  return jnp.concatenate([(state[:2] + jnp.pi) % (2 * jnp.pi) - jnp.pi, state[2:]])

def rk4_step(f, x, t, h):
  k1 = h * f(x, t)
  k2 = h * f(x + k1/2, t + h/2)
  k3 = h * f(x + k2/2, t + h/2)
  k4 = h * f(x + k3, t + h)
  return x + 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)

def radial2cartesian(t1, t2, l1, l2):
  x1 = l1 * jnp.sin(t1)
  y1 = -l1 * jnp.cos(t1)
  x2 = x1 + l2 * jnp.sin(t2)
  y2 = y1 - l2 * jnp.cos(t2)
  return x1, y1, x2, y2

def write_to(data, path):
  with open(path, 'wb') as f:
    pkl.dump(data, f, protocol=pkl.HIGHEST_PROTOCOL)

def read_from(path):
  with open(path, 'rb') as f:
    data = pkl.load(f)
  return data

def plot_loss(train_losses, test_losses):
  fig, ax = plt.subplots(1, 1)

  ax.plot(train_losses, label='Train loss')
  ax.plot(test_losses, label='Test loss')
  ax.set_ylabel("Loss")
  ax.set_xlabel("Epochs")
  ax.legend()

  fig.tight_layout()

# Lagrangian Dynamics

In [6]:
def unconstrained_eom(model, state, t=None):
  q, q_t = jnp.split(state, 2)
  return model(q, q_t)

def lagrangian_eom(lagrangian, state, t=None):
  q, q_t = jnp.split(state, 2)
  q = q % (2*jnp.pi)
  q_tt = (jnp.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t))
          @ (jax.grad(lagrangian, 0)(q, q_t)
             - jax.jacobian(jax.jacobian(lagrangian, 1), 0)(q, q_t) @ q_t))
  dt = 1e-1
  return dt*jnp.concatenate([q_t, q_tt])

def raw_lagrangian_eom(lagrangian, state, t=None):
  q, q_t = jnp.split(state, 2)
  q = q % (2*jnp.pi)
  q_tt = (jnp.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t))
          @ (jax.grad(lagrangian, 0)(q, q_t)
             - jax.jacobian(jax.jacobian(lagrangian, 1), 0)(q, q_t) @ q_t))
  return jnp.concatenate([q_t, q_tt])

def lagrangian_eom_rk4(lagrangian, state, n_updates, Dt=1e-1, t=None):
    @jax.jit
    def cur_fnc(state):
        q, q_t = jnp.split(state, 2)
        q = q % (2*jnp.pi)
        q_tt = (jnp.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t))
                 @ (jax.grad(lagrangian, 0)(q, q_t)
                 - jax.jacobian(jax.jacobian(lagrangian, 1), 0)(q, q_t) @ q_t))
        return jnp.concatenate([q_t, q_tt])

    @jax.jit
    def get_update(update):
        dt = Dt/n_updates
        cstate = state + update
        k1 = dt*cur_fnc(cstate)
        k2 = dt*cur_fnc(cstate + k1/2)
        k3 = dt*cur_fnc(cstate + k2/2)
        k4 = dt*cur_fnc(cstate + k3)
        return update + 1.0/6.0 * (k1 + 2*k2 + 2*k3 + k4)

    update = 0
    for _ in range(n_updates):
        update = get_update(update)
    return update


def solve_dynamics(dynamics_fn, initial_state, is_lagrangian=True, **kwargs):
  eom = lagrangian_eom if is_lagrangian else unconstrained_eom

  @partial(jax.jit, backend='cpu')
  def f(initial_state):
    return odeint(partial(eom, dynamics_fn), initial_state, **kwargs)
  return f(initial_state)


def custom_init(init_params, seed=0):
    import numpy as np
    new_params = []
    rng = jax.random.PRNGKey(seed)
    i = 0
    number_layers = len([0 for l1 in init_params if len(l1) != 0])
    for l1 in init_params:
        if (len(l1)) == 0: new_params.append(()); continue
        new_l1 = []
        for l2 in l1:
            if len(l2.shape) == 1:
                #Zero init biases
                new_l1.append(jnp.zeros_like(l2))
            else:
                n = max(l2.shape)
                first = int(i == 0)
                last = int(i == number_layers - 1)
                mid = int((i != 0) * (i != number_layers - 1))
                mid *= i

                std = 1.0/np.sqrt(n)
                std *= 2.2*first + 0.58*mid + n*last

                if std == 0:
                    raise NotImplementedError("Wrong dimensions for MLP")

                new_l1.append(jax.random.normal(rng, l2.shape)*std)
                rng += 1
                i += 1

        new_params.append(new_l1)

    return new_params

# Physics

In [7]:
@jit
def test_kinetic_energy(state, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15):
    q, q_dot = jnp.split(state, 2)
    t1, w1 = q, q_dot
    T = 0.5 * (I + m1*(d1**2))*(w1**2)
    return T

@jit
def test_potential_energy(state, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15):
    q, q_dot = jnp.split(state, 2)
    t1, w1 = q, q_dot
    V = -m1*g*d1*jnp.cos(t1)
    return V

@jit
def kinetic_energy(q, q_dot, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15):
  t1, w1 = q, q_dot
  T = 0.5 * (I + m1*(d1**2))*(w1**2)
  return T

@jit
def potential_energy(q, q_dot, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15):
  t1, w1 = q, q_dot
  V = -m1*g*d1*jnp.cos(t1)
  return V

@jit
def lagrangian_fn(q, q_dot, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15):
  T = kinetic_energy(q, q_dot)
  V = potential_energy(q, q_dot)
  return jnp.sum(T - V)

@jit
def analytical_fn(state, t=0, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15):
  t1, w1 = state
  a1 = 0
  f1 = -m1*g*d1*jnp.sin(t1)/(I+m1*(d1**2))
  g1 = f1
  return jnp.stack([w1,g1])

# Data

In [8]:
@partial(jax.jit, backend='cpu')
def get_trajectory(y0, times, use_lagrangian=False, **kwargs):
  if use_lagrangian:
    y = solve_dynamics(lagrangian_fn, y0, t=times, is_lagrangian=True, rtol=1e-10, atol=1e-10, **kwargs)
  else:
    y = odeint(analytical_fn, y0, t=times, rtol=1e-10, atol=1e-10, **kwargs)
  return y

@partial(jax.jit, backend='cpu')
def get_trajectory_lagrangian(y0, times, **kwargs):
  return solve_dynamics(lagrangian_fn, y0, t=times, is_lagrangian=True, rtol=1e-10, atol=1e-10, **kwargs)

@partial(jax.jit, backend='cpu')
def get_trajectory_analytic(y0, times, **kwargs):
    return odeint(analytical_fn, y0, t=times, rtol=1e-10, atol=1e-10, **kwargs)

def test_dataset(rng, samples=1, t_span=[0, 10], fps=100, test_split=0.5, lookahead=1,
                    unlimited_steps=False, init_pos=2.0*np.pi, init_vel=0.1, **kwargs):
    data = {'meta': locals()}
    vfnc = jax.jit(jax.vmap(analytical_fn))
    vget = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic, mxstep=100), (0, None), 0))
    vget_unlimited = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic), (0, None), 0))

    frames = int(fps*(t_span[1]-t_span[0]))
    times = jnp.linspace(t_span[0], t_span[1], frames)
    y0 = jnp.concatenate([
        jax.random.uniform(rng, (samples, 1))*init_pos,
        jax.random.uniform(rng+1, (samples, 1))*init_vel
    ], axis=1)

    if unlimited_steps:
      y = vget(y0, times)
    else:
      y = vget_unlimited(y0, times)

    data['x'] = y[:, :-lookahead]
    data['dx'] = y[:, lookahead:] - data['x']
    data['x'] = jnp.concatenate(data['x'])
    data['dx'] = jnp.concatenate(data['dx'])
    data['t'] = jnp.tile(times[:-lookahead], (samples,))

    split_ix = int(len(data['x']) * test_split)
    split_data = {}
    for k in ['x', 'dx', 't']:
        split_data[k], split_data['test_' + k] = data[k][:split_ix], data[k][split_ix:]
    data = split_data
    return data

def noise_test_dataset(rng, samples=1, t_span=[0, 10], fps=100, test_split=0.5, lookahead=1,
                    unlimited_steps=False, init_pos=2.0*np.pi, init_vel=0.1, **kwargs):
    data = {'meta': locals()}
    vfnc = jax.jit(jax.vmap(analytical_fn))
    vget = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic, mxstep=100), (0, None), 0))
    vget_unlimited = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic), (0, None), 0))

    frames = int(fps*(t_span[1]-t_span[0]))
    times = jnp.linspace(t_span[0], t_span[1], frames)
    y0 = jnp.concatenate([
        jax.random.uniform(rng, (samples, 1))*init_pos,
        jax.random.uniform(rng+1, (samples, 1))*init_vel
    ], axis=1)

    if unlimited_steps:
      y = vget(y0, times)
    else:
      y = vget_unlimited(y0, times)

    data['x'] = y[:, :-lookahead]
    data['x'] += jax.random.uniform(rng, data['x'].shape)*np.pi/2
    data['dx'] = y[:, lookahead:] - data['x']
    data['x'] = jnp.concatenate(data['x'])
    data['dx'] = jnp.concatenate(data['dx'])
    data['t'] = jnp.tile(times[:-lookahead], (samples,))

    split_ix = int(len(data['x']) * test_split)
    split_data = {}
    for k in ['x', 'dx', 't']:
        split_data[k], split_data['test_' + k] = data[k][:split_ix], data[k][split_ix:]
    data = split_data
    return data

# Test

In [ ]:
def test(lnn_path,baseline_path,exp_init_pos,exp_init_vel):
  max_t = 10
  fps = 10
  test_set = test_dataset(jax.random.PRNGKey(2),
                                t_span=[0, max_t],
                                fps=fps, test_split=1.0,
                                unlimited_steps=False,
                                init_pos=exp_init_pos,
                                init_vel=exp_init_vel)

  noise_set = noise_test_dataset(jax.random.PRNGKey(2),
                                t_span=[0, max_t],
                                fps=fps, test_split=1.0,
                                unlimited_steps=False,
                                init_pos=exp_init_pos,
                                init_vel=exp_init_vel)

  true_total_energy = (
      jax.vmap(test_kinetic_energy)(test_set['x'][:]) + \
      jax.vmap(test_potential_energy)(test_set['x'][:])
  )

  lnn_predictions = np.load(lnn_path[0])
  lnn_noise_predictions = np.load(lnn_path[1])
  lnn_total_energy = np.load(lnn_path[2])
  lnn_noise_total_energy = np.load(lnn_path[3])
  baseline_predictions = np.load(baseline_path[0])
  baseline_noise_predictions = np.load(baseline_path[1])
  baseline_total_energy = np.load(baseline_path[2])
  baseline_noise_total_energy = np.load(baseline_path[3])

  fig, ax = plt.subplots(2, 1, figsize=(10,10))

  fig.suptitle("Model Trajectory",fontsize=20)

  ax[0].set_title("Trajectory",fontsize=15)
  ax[0].plot(test_set['x'][:100, 0], label='Truth', color='blue')
  ax[0].set_ylabel('Configurations [rad]',rotation=0,fontsize=15)
  ax[0].set_xlabel(r'Time steps',fontsize=15)
  ax[0].legend()

  ax[1].set_title("Energy conservation",fontsize=15)
  ax[1].plot((true_total_energy),label='Truth', color='blue')
  ax[1].set_ylim([-0.5,0.5])
  ax[1].set_ylabel('Total Energy [J]',fontsize=15)
  ax[1].set_xlabel(r'Time steps',fontsize=15)
  ax[1].legend()
  
  fig.tight_layout()

  fig, ax = plt.subplots(3, 1, figsize=(10,10))
  
  fig.suptitle("Free evolution without noise",fontsize=20)

  ax[0].set_title("Trajectory",fontsize=15)
  ax[0].plot(test_set['x'][:100, 0], label='Truth', color='blue')
  ax[0].plot(lnn_predictions[:100, 0], label='LNN', color='green')
  ax[0].plot(baseline_predictions[:100, 0], label='Baseline', color='orange')
  ax[0].set_ylabel('Configuration [rad]',rotation=0,fontsize=15)
  ax[0].set_xlabel(r'Time steps',fontsize=15)
  ax[0].legend()

  ax[1].set_title("Trajectory error",fontsize=15)
  ax[1].plot(jnp.abs(-test_set['x'][:100, 0] + lnn_predictions[:100, 0]), label='LNN', color='green')
  ax[1].plot(jnp.abs(-test_set['x'][:100, 0] + baseline_predictions[:100, 0]), label='Baseline', color='orange')
  ax[1].set_ylabel('Error [rad]',fontsize=15)
  ax[1].set_xlabel(r'Time steps',fontsize=15)
  ax[1].legend()

  lnn_rmse = jnp.sqrt(jnp.mean((-test_set['x'][:100, 0] + lnn_predictions[:100, 0])**2))
  baseline_rmse = jnp.sqrt(jnp.mean((-test_set['x'][:100, 0] + baseline_predictions[:100, 0])**2))
  print("Trajectory RMSE q0={} qdot0={}: lnn = {} [rad], baseline = {} [rad]".format(exp_init_pos,exp_init_vel,lnn_rmse,baseline_rmse))

  ax[2].set_title("Energy conservation",fontsize=15)
  ax[2].plot((lnn_total_energy[:100]),label='LNN', color='green')
  ax[2].plot((true_total_energy[:100]),label='Truth', color='blue')
  ax[2].plot((baseline_total_energy[:100]),label='Baseline', color='orange')
  ax[2].set_ylabel('Total Energy [J]',fontsize=15)
  ax[2].set_xlabel(r'Time steps',fontsize=15)
  ax[2].legend()

  fig.tight_layout()

  fig, ax = plt.subplots(3, 1, figsize=(10,10))

  fig.suptitle("Free evolution with noise",fontsize=20)

  ax[0].set_title("Trajectory",fontsize=15)
  ax[0].plot(test_set['x'][:100, 0], label='Truth', color='blue')
  ax[0].plot(noise_set['x'][:100, 0], label='Measured', color='violet')
  ax[0].plot(lnn_noise_predictions[:100, 0], label='LNN', color='green')
  ax[0].plot(baseline_noise_predictions[:100, 0], label='Baseline', color='orange')
  ax[0].set_ylabel('Configuration [rad]',rotation=0,fontsize=15)
  ax[0].set_xlabel(r'Time steps',fontsize=15)
  ax[0].legend()
  
  lnn_noise_rmse = jnp.sqrt(jnp.mean((-test_set['x'][:100, 0] + lnn_noise_predictions[:100, 0])**2))
  baseline_noise_rmse = jnp.sqrt(jnp.mean((-test_set['x'][:100, 0] + baseline_noise_predictions[:100, 0])**2))
  print("Noise trajectory RMSE q0={} qdot0={}: lnn = {} [rad], baseline = {} [rad]".format(exp_init_pos,exp_init_vel,lnn_noise_rmse,baseline_noise_rmse))
  
  ax[1].set_title("Trajectory error",fontsize=15)
  ax[1].plot(jnp.abs(-test_set['x'][:100, 0] + lnn_noise_predictions[:100, 0]), label='LNN', color='green')
  ax[1].plot(jnp.abs(-test_set['x'][:100, 0] + baseline_noise_predictions[:100, 0]), label='Baseline', color='orange')
  ax[1].set_ylabel('Error [rad]',fontsize=15)
  ax[1].set_xlabel(r'Time steps',fontsize=15)
  ax[1].legend()

  ax[2].set_title("Energy conservation",fontsize=15)
  ax[2].plot((lnn_noise_total_energy),label='LNN', color='green')
  ax[2].plot((true_total_energy),label='Truth', color='blue')
  ax[2].plot((baseline_noise_total_energy),label='Baseline', color='orange')
  ax[2].set_ylabel('Total Energy [J]',fontsize=15)
  ax[2].set_xlabel(r'Time steps',fontsize=15)
  ax[2].legend()

  fig.tight_layout()

init_pos = 2.0*np.pi
init_vel = 0.5

lnn_path = [root_path + "Rigid/LNN/lnn_" + str(round(init_pos,2)) + "_" + str(round(init_vel,2)) + "_rigid_predictions.npy", #colab
            root_path + "Rigid/LNN/lnn_" + str(round(init_pos,2)) + "_" + str(round(init_vel,2)) + "_rigid_noise_predictions.npy",
            root_path + "Rigid/LNN/lnn_" + str(round(init_pos,2)) + "_" + str(round(init_vel,2)) + "_rigid_energy.npy"]

baseline_path = [root_path + "Rigid/Baseline/baseline_nn_" + str(round(init_pos,2)) + "_" + str(round(init_vel,2)) + "_rigid_predictions.npy", #colab
                 root_path + "Rigid/Baseline/baseline_nn_" + str(round(init_pos,2)) + "_" + str(round(init_vel,2)) + "_rigid_noise_predictions.npy",
                 root_path + "Rigid/Baseline/baseline_nn_" + str(round(init_pos,2)) + "_" + str(round(init_vel,2)) + "_rigid_energy.npy"]

test(lnn_path,baseline_path,init_pos,init_vel)